In [1]:
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
import numpy as np

In [2]:
username = "icekhan"
version = "latest"

In [3]:
def training_op(**kwargs):
    return dsl.ContainerOp(
        name="training-operator",
        image=f"{username}/mock_training:{version}",
        arguments=["--data-size", kwargs["data_size"]]
    )

def evaluation_op():
    return dsl.ContainerOp(
        name="evaluation-operator",
        image=f"{username}/mock_evaluation:{version}"
    )

def plot_op():
    return dsl.ContainerOp(
        name="plotting-operator",
        image=f"{username}/mock_plot:{version}"
    )

In [4]:
@dsl.pipeline(name="Sapsan demp pipeline", description="Pipeline description")
def pipeline_definition(data_size):
    train = training_op(data_size=data_size)
    evaluate = evaluation_op().after(train)
    plot = plot_op().after(evaluate)

In [5]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")


In [6]:
client = kfp.Client()

Failed to load kube config.


In [ ]:
%%bash
tar -xvf pipeline.tar.gz


In [ ]:
run_name = f"Run number {np.random.randint(0, 10000)}"
experiment_name='Sapsan experiment'
# get or create an experiment_id
try:
    experiment_id = client.get_experiment(experiment_name=experiment_name).id
except:
    experiment_id = client.create_experiment(experiment_name).id
experiment_id

In [ ]:
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "data_size": 10,
    }
)